In [1]:
import os
from Powerfleet_API_Management import PowerfleetAPIManager as powerfleet_API
import pandas as pd
import json
import csv


print(f"{os.getcwd()}")
print(f"{os.listdir()}")

/home/georger/WorkDocuments/Sxoli/Ptyxiaki/Ptyxiaki-Repository/Python_Scripts/Data_Analysis
os.getcwd()='/home/georger/WorkDocuments/Sxoli/Ptyxiaki/Ptyxiaki-Repository/Python_Scripts/Data_Analysis'
['Powerfleet_API_Management.py', 'JSON_response.json', '__pycache__', 'data_analysis.ipynb', 'Powerfleet_API_CredentialsI.txt']
Headers: {'Content-Type': 'application/json', 'Authorization': '00595c0d-01d4-38a3-ba06-8852a0c6b2b8'}
Data: {'startDate': '2024-10-23 15:32:24', 'endDate': '2024-11-25 15:32:24', 'vehicleId': 7, 'cid': '946'}
Success: 200

[{'vehicleId': 7, 'lat': 37.508455, 'lng': 22.3740133, 'dateStored': 1729707515000, 'velocity': 0.0, 'odometer': 0.0, 'engineVoltage': 4.762}, {'vehicleId': 7, 'lat': 37.50836, 'lng': 22.3737299, 'dateStored': 1729707516000, 'velocity': 19.0, 'odometer': 0.0, 'engineVoltage': 4.762}, {'vehicleId': 7, 'lat': 37.508375, 'lng': 22.3736733, 'dateStored': 1729707528000, 'velocity': 7.0, 'odometer': 0.0, 'engineVoltage': 4.756}, {'vehicleId': 7, 'lat':

Convert **JSON** to **CSV** so that **pandas** be able to use it


In [2]:
api = powerfleet_API() # get data

with open('JSON_response.json', 'w') as json_file:
    json_file.write(str(api.retrieve_response()))


os.getcwd()='/home/georger/WorkDocuments/Sxoli/Ptyxiaki/Ptyxiaki-Repository/Python_Scripts/Data_Analysis'
['Powerfleet_API_Management.py', 'JSON_response.json', '__pycache__', 'data_analysis.ipynb', 'Powerfleet_API_CredentialsI.txt']
Headers: {'Content-Type': 'application/json', 'Authorization': '00595c0d-01d4-38a3-ba06-8852a0c6b2b8'}
Data: {'startDate': '2024-10-23 15:32:24', 'endDate': '2024-11-25 15:32:24', 'vehicleId': 7, 'cid': '946'}
Success: 200

